In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision as torchvision
from torchvision import datasets as datasets
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from model import Discriminator,Generator,initialize_weights



ModuleNotFoundError: No module named 'model'

In [21]:
# Define device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LEARNING_RATE=2e-4
Batch_size=128
image_size=64
channels_img=1
z_dim=100
num_epochs=10
features_disc=64
features_gen=64
transforms=transforms.Compose(
    [
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.5 for _ in range(channels_img)],[0.5 for _ in range(channels_img)]
        ),

    ]

)
dataset=datasets.MNIST(root="dataset/",train=True,transform=transforms,download=True)
loader=DataLoader(dataset,batch_size=Batch_size,shuffle=True)
gen=Generator(z_dim,channels_img,features_gen).to(device)
disc=Discriminator(channels_img,features_disc).to(device)
initialize_weights(gen)
initialize_weights(disc)
opt_gen=optim.Adam(gen.parameters(),lr=LEARNING_RATE,betas=(0.5,0.999))
opt_disc=optim.Adam(disc.parameters(),lr=LEARNING_RATE,betas=(0.5,0.999))
criterion=nn.BCELoss()
fixed_noise=torch.randn(32,z_dim,1,1).to(device)
writer_real=SummaryWriter(f"logs/real")
writer_fake=SummaryWriter(f"logs/fake")
step=0
gen.train()
disc.train()


Discriminator(
  (disc): Sequential(
    (0): Sequential(
      (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (1): Sequential(
      (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (2): Sequential(
      (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (3): Sequential(
      (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): 

In [23]:
for epoch in range(num_epochs):
  for batch_idx,(real,_) in enumerate(loader):
    real=real.to(device)
    noise=torch.randn(Batch_size,z_dim,1,1).to(device)
    fake=gen(noise)

    #train_discriminator
    noise=torch.randn(Batch_size,z_dim).to(device)

    disc_real=disc(real).view(-1)
    lossD_real=criterion(disc_real,torch.ones_like(disc_real))
    disc_fake=disc(fake).view(-1)
    lossD_fake=criterion(disc_fake,torch.zeros_like(disc_fake))
    lossD=(lossD_fake+lossD_real)/2
    disc.zero_grad()
    lossD.backward(retain_graph=True)
    opt_disc.step()


    #train_generator
    output=disc(fake).view(-1)
    lossG=criterion(output,torch.ones_like(output))
    gen.zero_grad()
    lossG.backward()
    opt_gen.step()


    if batch_idx==0:
       print(f"Epochs[{epoch}\ {num_epochs}]/"
             f"Loss D:{lossD:.4f},Loss G:{lossG:.4f}")
       with torch.no_grad():
         fake=gen(fixed_noise)

         img_grid_fake=torchvision.utils.make_grid(fake[:32],normalize=True)
         img_grid_real=torchvision.utils.make_grid(real[:32],normalize=True)


         writer_fake.add_image(
            "MNIST fake images",img_grid_fake,global_step=step
        )
         writer_real.add_image(
            "MNIST real images",img_grid_real,global_step=step
        )
         step+=1



Epochs[0\ 10]/Loss D:6.8022,Loss G:5.4784


KeyboardInterrupt: 